# Bedrock Agent Core with MCP Gateway Integration - Complete Guide

This notebook demonstrates a complete end-to-end implementation of AWS Bedrock Agent Core integrated with MCP Gateway for intelligent research report generation and structured PDF output.

## Architecture Overview

```
User Query → Bedrock Agent Core → Internet Search → Qwen Analysis → MCP Gateway → Lambda PDF Generator → S3 Storage
```

## Components

1. **Bedrock Agent Core Runtime**: Main research agent with Qwen integration
2. **Internet Search Tool**: Real-time information gathering
3. **Qwen Model Integration**: AI-powered analysis via SageMaker endpoints
4. **MCP Gateway**: Secure Lambda function orchestration
5. **Markdown Generation Lambda**: Structured report creation
6. **S3 Storage**: Persistent report storage

## Tutorial Details

| Information          | Details                                                   |
|:---------------------|:----------------------------------------------------------|
| Tutorial type        | Interactive End-to-End                                    |
| AgentCore components | AgentCore Runtime + AgentCore Gateway                     |
| Agentic Framework    | Bedrock Agent Core with LangChain                         |
| Gateway Target type  | AWS Lambda (PDF Generation)                               |
| Inbound Auth         | AWS IAM                                                   |
| Outbound Auth        | AWS IAM                                                   |
| LLM model            | Qwen via SageMaker Endpoint                               |
| Tutorial components  | Research Agent + MCP Gateway + PDF Generation             |
| Tutorial vertical    | Research and Document Generation                          |
| Example complexity   | Advanced                                                  |
| SDK used             | boto3, bedrock-agentcore-starter-toolkit                 |

## Prerequisites

- SageMaker Studio environment
- AWS CLI configured with appropriate permissions
- Python 3.8+ environment
- Qwen model deployed on SageMaker endpoint
- Docker support in SageMaker Studio

## 1. Environment Setup and Dependencies

First, we'll install all required dependencies and verify our environment setup.

In [ ]:
%pip install --force-reinstall -U --quiet --no-warn-conflicts  -r requirements.txt

In [ ]:
# Import required libraries
import boto3
import json
import os
import time
from datetime import datetime
from bedrock_agentcore_starter_toolkit import Runtime
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.runnables import Runnable

# Set up AWS clients
session = boto3.Session()
region = session.region_name or 'us-east-1'
account_id = boto3.client('sts').get_caller_identity()['Account']

print(f"AWS Region: {region}")
print(f"Account ID: {account_id}")
print(f"Session configured successfully")

You can use AWS CLI to find a name for the endpoint we deployed in Lab1. For more information about AWS CLI please refer to the [documentation](https://docs.aws.amazon.com/cli/).

As an alternative, you can also use `boto3` API (AWS SDK for Python) to get a list of available endpoints. See AWS SDK for Python [documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) for details.

In this workshop we will use AWS CLI.

In [ ]:
endpoint_list = !aws sagemaker list-endpoints --status-equals "InService" --query 'Endpoints[*].EndpointName' --output text

In [ ]:
ENDPOINT = endpoint_list[0]
print(ENDPOINT)

## 2. IAM Role Creation

Create a comprehensive IAM role that includes all necessary permissions for Bedrock Agent Core, SageMaker endpoint access, MCP Gateway integration, and other AWS services.

**Please note that if you are running this notebook as part of AWS workshop the role `BedrockAgentCoreQwenRole` was already created as part of the workshop setup.**
You should see the following message: 
```
Role BedrockAgentCoreQwenRole already exists: arn:aws:iam::ACCOUNT_ID:role/BedrockAgentCoreQwenRole
```

If you run this notebook in your own account please make sure you have permissions to create an IAM role.

In [ ]:
def create_bedrock_agentcore_role():
    iam_client = boto3.client('iam')
    
    role_name = 'BedrockAgentCoreQwenRole'
    
    trust_policy = {
        'Version': '2012-10-17',
        'Statement': [
            {
                'Effect': 'Allow',
                'Principal': {
                    'Service': 'bedrock-agentcore.amazonaws.com'
                },
                'Action': 'sts:AssumeRole'
            },
            {
                'Effect': 'Allow',
                'Principal': {
                    'Service': 'lambda.amazonaws.com'
                },
                'Action': 'sts:AssumeRole'
            }
        ]
    }
    
    permissions_policy = {
        'Version': '2012-10-17',
        'Statement': [
            {
                'Effect': 'Allow',
                'Action': [
                    'sagemaker:InvokeEndpoint',
                    'bedrock:InvokeModel',
                    'bedrock:InvokeModelWithResponseStream',
                    'logs:CreateLogGroup',
                    'logs:CreateLogStream',
                    'logs:PutLogEvents',
                    'lambda:InvokeFunction',
                    's3:GetObject',
                    's3:PutObject',
                    's3:DeleteObject',
                    'ecr:GetAuthorizationToken',
                    'ecr:BatchGetImage',
                    'ecr:GetDownloadUrlForLayer',
                    'ecr:BatchCheckLayerAvailability'
                ],
                'Resource': '*'
            }
        ]
    }
    
    try:
        role_response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description='Role for Bedrock Agent Core with Qwen integration'
        )
        
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName='BedrockAgentCoreQwenPolicy',
            PolicyDocument=json.dumps(permissions_policy)
        )
        
        role_arn = role_response['Role']['Arn']
        print('✓ Created IAM role: ' + role_arn)
        
    except iam_client.exceptions.EntityAlreadyExistsException:
        role_arn = 'arn:aws:iam::' + account_id + ':role/' + role_name
        print('✓ Using existing IAM role: ' + role_arn)
    
    return role_arn

In [ ]:
bedrock_role_arn = create_bedrock_agentcore_role()

## 3. Core Components Setup

Create the core components: internet search tool, and research agent.

In [ ]:
# Create agent directory and tools
import os
os.makedirs('agent', exist_ok=True)

# Create agent/__init__.py
with open('agent/__init__.py', 'w') as f:
    f.write('')

# Create agent/tools.py with REAL working links
tools_code = '''
import requests
import json
from typing import List, Dict, Any
from urllib.parse import quote_plus

def internet_search(query: str, max_results: int = 5) -> Dict[str, Any]:
    """Search that returns REAL working Wikipedia links"""
    
    try:
        # Use Wikipedia API to get real articles
        search_url = "https://en.wikipedia.org/w/api.php"
        search_params = {
            'action': 'query',
            'format': 'json',
            'list': 'search',
            'srsearch': query,
            'srlimit': max_results
        }
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (compatible; ResearchBot/1.0)'
        }
        
        response = requests.get(search_url, params=search_params, headers=headers, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            results = []
            
            for page in data.get('query', {}).get('search', []):
                title = page['title']
                snippet = page.get('snippet', '').replace('<span class="searchmatch">', '').replace('</span>', '')
                
                # Create REAL Wikipedia URL
                wiki_url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"
                
                results.append({
                    'title': title,
                    'content': snippet + f" - Wikipedia article about {title}",
                    'url': wiki_url,
                    'source': 'Wikipedia'
                })
            
            return {
                'query': query,
                'results': results,
                'total_results': len(results)
            }
    
    except Exception as e:
        pass
    
    # Fallback with real URLs
    fallback_results = [
        {
            'title': f'Wikipedia: {query}',
            'content': f'Encyclopedia article about {query}',
            'url': f'https://en.wikipedia.org/wiki/{query.replace(" ", "_")}',
            'source': 'Wikipedia'
        },
        {
            'title': f'Research: {query}',
            'content': f'Academic research and information about {query}',
            'url': f'https://en.wikipedia.org/wiki/Main_Page',
            'source': 'Wikipedia'
        }
    ]
    
    return {
        'query': query,
        'results': fallback_results[:max_results],
        'total_results': len(fallback_results[:max_results])
    }
'''

with open('agent/tools.py', 'w') as f:
    f.write(tools_code)

print("✓ Created tools.py with REAL Wikipedia links")


## 4. MCP Gateway Lambda Function

Create and deploy the Lambda function that will serve as our MCP server for PDF generation.

In [ ]:
# Create the Lambda MCP server for PDF generation
lambda_mcp_code = '''
import json
import boto3
from datetime import datetime

def lambda_handler(event, context):
    """Lambda MCP server for research report processing"""
    
    try:
        body = json.loads(event.get('body', '{}'))
        method = body.get('method')
        params = body.get('params', {})
    except:
        return {
            'statusCode': 400,
            'body': json.dumps({'error': 'Invalid JSON'})
        }
    
    if method == 'tools/list':
        return {
            'statusCode': 200,
            'body': json.dumps({
                "tools": [
                    {
                        "name": "save_research_report",
                        "description": "Save research report as structured document to S3",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "content": {"type": "string", "description": "Report content"},
                                "filename": {"type": "string", "description": "File name"},
                                "bucket": {"type": "string", "description": "S3 bucket name"},
                                "format": {"type": "string", "description": "Output format (markdown/json)", "default": "markdown"}
                            },
                            "required": ["content", "filename", "bucket"]
                        }
                    }
                ]
            })
        }
    
    elif method == 'tools/call':
        tool_name = params.get('name')
        args = params.get('arguments', {})
        
        if tool_name == 'save_research_report':
            try:
                result = save_structured_report(
                    args['content'],
                    args['filename'], 
                    args['bucket'],
                    args.get('format', 'markdown')
                )
                
                return {
                    'statusCode': 200,
                    'body': json.dumps({
                        "content": [{
                            "type": "text",
                            "text": result
                        }]
                    })
                }
            except Exception as e:
                return {
                    'statusCode': 500,
                    'body': json.dumps({
                        "content": [{
                            "type": "text", 
                            "text": f"Error: {str(e)}"
                        }]
                    })
                }
    
    return {
        'statusCode': 400,
        'body': json.dumps({'error': 'Unknown method'})
    }

def save_structured_report(content, filename, bucket, format_type='markdown'):
    """Save structured research report to S3"""
    s3_client = boto3.client('s3')
    
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S UTC")
    
    if format_type == 'json':
        # Structure as JSON
        structured_content = {
            "metadata": {
                "generated": timestamp,
                "type": "research_report",
                "format": "structured_json"
            },
            "content": content,
            "processing_info": {
                "processor": "AgentCore MCP Gateway",
                "lambda_function": "research-report-processor"
            }
        }
        
        body = json.dumps(structured_content, indent=2).encode('utf-8')
        content_type = 'application/json'
        
    else:
        # Structure as enhanced markdown
        structured_content = f"""---
title: Research Report
generated: {timestamp}
type: research_report
processor: AgentCore MCP Gateway
format: structured_markdown
---

{content}

---

## Processing Information

- **Generated**: {timestamp}
- **Processor**: AgentCore MCP Gateway
- **Lambda Function**: research-report-processor
- **Format**: Structured Markdown with YAML frontmatter

This report was automatically generated and processed through AWS Bedrock Agent Core with MCP Gateway integration.
"""
        
        body = structured_content.encode('utf-8')
        content_type = 'text/markdown'
    
    # Upload to S3
    s3_client.put_object(
        Bucket=bucket,
        Key=filename,
        Body=body,
        ContentType=content_type,
        Metadata={
            'generated-by': 'agentcore-mcp-gateway',
            'report-type': 'research-analysis',
            'timestamp': timestamp
        }
    )
    
    return f"Structured report saved successfully to s3://{bucket}/{filename} (format: {format_type})"
'''

with open('lambda_mcp_server.py', 'w') as f:
    f.write(lambda_mcp_code)

print("✓ Created lambda_mcp_server.py")

## 5. Deploy Lambda MCP Server

Deploy the Lambda function that will serve as our MCP server.

In [ ]:
import zipfile
import tempfile
import shutil

def deploy_lambda_mcp_server():
    """Deploy Lambda MCP server"""
    lambda_client = boto3.client('lambda')
    iam_client = boto3.client('iam')
    
    # Create Lambda execution role
    role_name = 'ResearchReportMCPRole'
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {"Service": "lambda.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }]
    }
    
    try:
        role_response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy)
        )
        role_arn = role_response['Role']['Arn']
        
        # Attach policies
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )
        
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
        )
        
        print(f"✓ Created Lambda execution role: {role_arn}")
        
    except iam_client.exceptions.EntityAlreadyExistsException:
        role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"
        print(f"✓ Using existing Lambda role: {role_arn}")
    
    # Create deployment package
    zip_path = 'lambda_mcp_deployment.zip'
    with zipfile.ZipFile(zip_path, 'w') as zip_file:
        zip_file.write('lambda_mcp_server.py', 'lambda_function.py')
    
    # Deploy Lambda function
    function_name = 'research-report-mcp-server'
    
    # Wait for role propagation
    print("Waiting for IAM role propagation...")
    time.sleep(30)
    
    try:
        with open(zip_path, 'rb') as zip_file:
            lambda_client.create_function(
                FunctionName=function_name,
                Runtime='python3.12',
                Role=role_arn,
                Handler='lambda_function.lambda_handler',
                Code={'ZipFile': zip_file.read()},
                Timeout=60,
                MemorySize=512,
                Description='MCP server for research report processing'
            )
        print(f"✓ Lambda function created: {function_name}")
        
    except lambda_client.exceptions.ResourceConflictException:
        with open(zip_path, 'rb') as zip_file:
            lambda_client.update_function_code(
                FunctionName=function_name,
                ZipFile=zip_file.read()
            )
        print(f"✓ Lambda function updated: {function_name}")
    
    # Get function ARN
    response = lambda_client.get_function(FunctionName=function_name)
    function_arn = response['Configuration']['FunctionArn']
    
    # Cleanup
    os.remove(zip_path)
    
    return function_arn

# Deploy the Lambda MCP server
lambda_mcp_arn = deploy_lambda_mcp_server()
print(f"\nLambda MCP Server ARN: {lambda_mcp_arn}")

## 6. Create MCP Gateway Integration

Create the integration layer that connects our research agent with the MCP Gateway.

In [ ]:
# Create MCP Gateway client
mcp_client_code = '''
import json
import boto3
from datetime import datetime

class MCPGatewayClient:
    """Client for AgentCore MCP Gateway integration"""
    
    def __init__(self, lambda_function_name='research-report-mcp-server'):
        self.lambda_client = boto3.client('lambda')
        self.function_name = lambda_function_name
    
    def call_mcp_tool(self, tool_name, arguments):
        """Call MCP tool via Lambda function"""
        payload = {
            "method": "tools/call",
            "params": {
                "name": tool_name,
                "arguments": arguments
            }
        }
        
        try:
            response = self.lambda_client.invoke(
                FunctionName=self.function_name,
                Payload=json.dumps({
                    'body': json.dumps(payload)
                })
            )
            
            result = json.loads(response['Payload'].read())
            return json.loads(result['body'])
            
        except Exception as e:
            return {
                "content": [{
                    "type": "text",
                    "text": f"MCP Gateway Error: {str(e)}"
                }]
            }
    
    def save_research_report(self, content, bucket_name, format_type='markdown'):
        """Save research report using MCP Gateway"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"research_report_{timestamp}.{format_type}"
        
        result = self.call_mcp_tool(
            "save_research_report",
            {
                "content": content,
                "filename": filename,
                "bucket": bucket_name,
                "format": format_type
            }
        )
        
        return result
'''

with open('mcp_gateway_client.py', 'w') as f:
    f.write(mcp_client_code)

print("✓ Created mcp_gateway_client.py")

## 7. Create Integrated Research Agent

Create the main research agent that integrates Qwen, internet search, and MCP Gateway.

In [ ]:
def create_main_function(bucket_name, endpoint_name="model-251111-2330"):
    main_app_code = f'''from bedrock_agentcore.runtime import BedrockAgentCoreApp
import json
import boto3
from agent.tools import internet_search
import requests
from bs4 import BeautifulSoup
import io

app = BedrockAgentCoreApp()

# Get account info
account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name or 'us-west-2'

class LineIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord("\\n"):
                self.read_pos += len(line)
                return line[:-1]
            try:
                chunk = next(self.byte_iterator)
            except StopIteration:
                if self.read_pos < self.buffer.getbuffer().nbytes:
                    continue
                raise
            if "PayloadPart" not in chunk:
                continue
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk["PayloadPart"]["Bytes"])

def fetch_content_from_url(url):
    """Fetch actual content from Wikipedia URLs"""
    try:
        headers = {{
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }}
        
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove unwanted elements
        for element in soup(['script', 'style', 'nav', 'header', 'footer', '.navbox', '.infobox']):
            element.decompose()
        
        # Get main content
        content_div = soup.find('div', {{'id': 'mw-content-text'}})
        if content_div:
            text = content_div.get_text()
        else:
            text = soup.get_text()
        
        # Clean up text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        # Return first 2000 characters
        return text[:2000] if text else "No content extracted"
        
    except Exception as e:
        return f"Could not fetch content: {{str(e)}}"

def stream_response(endpoint_name, inputs, max_tokens=1000, temperature=0.3, top_p=0.8):
    """Use SageMaker streaming inference"""
    try:
        smr_client = boto3.client('sagemaker-runtime', region_name=region)
        
        body = {{
          "messages": [
            {{"role": "user", "content": [{{"type": "text", "text": inputs}}]}}
            ],
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "stream": True,
        }}

        resp = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(body),
            ContentType="application/json",
        )

        event_stream = resp["Body"]
        start_json = b"{{"
        full_response = ""

        for line in LineIterator(event_stream):
            if line != b"" and start_json in line:
                data = json.loads(line[line.find(start_json):].decode("utf-8"))
                token_text = data['choices'][0]['delta'].get('content', '')
                full_response += token_text

        return full_response
        
    except Exception as e:
        return f"Model analysis error: {{str(e)}}"

def call_mcp_gateway(content, bucket):
    """Call MCP Gateway Lambda to save report"""
    lambda_client = boto3.client('lambda')
    
    payload = {{
        "method": "tools/call",
        "params": {{
            "name": "save_research_report",
            "arguments": {{
                "content": content,
                "filename": f"research_report_{{int(account_id)}}.md",
                "bucket": bucket,
                "format": "markdown"
            }}
        }}
    }}
    
    try:
        response = lambda_client.invoke(
            FunctionName='research-report-mcp-server',
            Payload=json.dumps({{'body': json.dumps(payload)}})
        )
        
        result = json.loads(response['Payload'].read())
        return json.loads(result['body'])
    except Exception as e:
        return {{"content": [{{"type": "text", "text": f"MCP Error: {{str(e)}}"}}]}}

@app.entrypoint
async def simple_qwen_research(payload):
    """Complete research pipeline: tools.py -> fetch content -> model analysis -> MCP report"""
    user_input = payload.get("prompt", "test query")
    
    # Step 1: Use tools.py to get search results
    search_results = internet_search(user_input, max_results=3)
    
    # Step 2: Fetch actual content from the URLs returned by tools.py
    enriched_results = []
    all_content = ""
    
    for result in search_results.get('results', []):
        url = result.get('url', '')
        title = result.get('title', '')
        source = result.get('source', '')
        
        print(f"Fetching content from: {{url}}")
        actual_content = fetch_content_from_url(url)
        
        enriched_results.append({{
            'title': title,
            'source': source,
            'url': url,
            'content': actual_content
        }})
        
        all_content += f"\\n\\nSource: {{source}} - {{title}}\\n{{actual_content}}"
    
    # Step 3: Use SageMaker model to analyze the content
    if all_content.strip():
        prompt = f"""Analyze this research content about "{{user_input}}" and provide:

SUMMARY: [Write a 2-3 sentence summary of the key findings]
KEY POINTS:
• [First key finding]
• [Second key finding] 
• [Third key finding]
TRENDS: [Notable trends or developments mentioned]

Content to analyze:
{{all_content[:3000]}}"""

        model_analysis = stream_response("{endpoint_name}", prompt)
    else:
        model_analysis = "SUMMARY: No content available for analysis. KEY POINTS: • No sources found TRENDS: Unable to determine trends."
    
    # Step 4: Parse model analysis
    summary = "Analysis completed"
    key_points = []
    trends = "Current developments noted"
    
    if "SUMMARY:" in model_analysis:
        parts = model_analysis.split("SUMMARY:")[1]
        if "KEY POINTS:" in parts:
            summary = parts.split("KEY POINTS:")[0].strip()
            key_section = parts.split("KEY POINTS:")[1]
            if "TRENDS:" in key_section:
                key_text = key_section.split("TRENDS:")[0]
                trends = key_section.split("TRENDS:")[1].strip()
            else:
                key_text = key_section
            
            key_points = [line.strip() for line in key_text.split('\\n') if line.strip().startswith('•')]
    
    # Step 5: Generate comprehensive report
    sources_section = ""
    detailed_findings = ""
    
    for i, result in enumerate(enriched_results, 1):
        title = result.get('title', 'N/A')
        url = result.get('url', 'N/A')
        source = result.get('source', 'Unknown')
        content = result.get('content', 'No content available')
        
        sources_section += f"{{i}}. **{{source}}**: [{{title}}]({{url}})\\n"
        
        # Show actual content excerpt
        if content and len(content) > 100:
            excerpt = content[:400] + "..." if len(content) > 400 else content
            detailed_findings += f"**{{title}}** ({{source}})\\n{{excerpt}}\\n\\n"
    
    key_points_section = "\\n".join(key_points) if key_points else "• Analysis completed with available sources"
    
    report = f"""# Research Report: {{user_input}}

## Executive Summary
{{summary}}

## Sources Analyzed (via tools.py)
{{sources_section}}

## Key Findings & Insights
{{key_points_section}}

## Trends & Developments
{{trends}}

## Detailed Source Analysis
{{detailed_findings}}

## Research Pipeline
- **Step 1**: tools.py internet search
- **Step 2**: Content extraction from URLs
- **Step 3**: SageMaker model analysis ({{"{endpoint_name}"}})
- **Step 4**: MCP Gateway S3 storage

---
*Complete Agent Core tutorial: tools.py → content fetch → model analysis → MCP report*"""

    # Step 6: Use MCP Gateway to save report to S3
    mcp_result = call_mcp_gateway(report, "{bucket_name}")
    storage_status = mcp_result.get('content', [{{}}])[0].get('text', 'MCP processing completed')
    
    final_report = report + f"\\n\\n## MCP Gateway Status\\n{{storage_status}}"
    
    yield {{
        "messages": [{{
            "type": "assistant", 
            "content": final_report
        }}]
    }}

if __name__ == "__main__":
    app.run()'''
    
    return main_app_code

# Example usage:
bucket_name = f"sagemaker-{region}-{account_id}"
endpoint_name = ENDPOINT
main_code = create_main_function(bucket_name, endpoint_name)

with open('main.py', 'w') as f:
    f.write(main_code)

print(f"✓ Created complete tutorial main.py: tools.py → content fetch → SageMaker model → MCP report")


## 8. Local Testing

Test the integrated system locally before deploying to Bedrock Agent Core.

In [ ]:
# Test the MCP Gateway integration locally
from mcp_gateway_client import MCPGatewayClient

def test_mcp_gateway():
    """Test MCP Gateway functionality"""
    print("Testing MCP Gateway Integration...")
    print("=" * 50)
    
    mcp_client = MCPGatewayClient()
    
    # Test content
    test_content = """# Test Research Report

## Overview
This is a test report to verify MCP Gateway integration with Bedrock Agent Core.

## Key Findings
- MCP Gateway successfully deployed
- Lambda function processing reports
- S3 integration working correctly
- Structured output generation functional

## Conclusion
The integrated system is ready for production deployment with Bedrock Agent Core.
"""
    
    # Test S3 bucket
    # test_bucket = f"bedrock-agentcore-codebuild-{account_id}-{region}"
    test_bucket = f"sagemaker-{region}-{account_id}"
    
    try:
        # Test markdown format
        result_md = mcp_client.save_research_report(test_content, test_bucket, 'markdown')
        print("Markdown Test Result:")
        print(json.dumps(result_md, indent=2))
        
        # Test JSON format
        result_json = mcp_client.save_research_report(test_content, test_bucket, 'json')
        print("\nJSON Test Result:")
        print(json.dumps(result_json, indent=2))
        
        print("\n✓ MCP Gateway integration test successful!")
        return True
        
    except Exception as e:
        print(f"✗ MCP Gateway test failed: {str(e)}")
        return False

# Run MCP Gateway test
mcp_test_success = test_mcp_gateway()

In [ ]:
# Test the complete integrated research system
import asyncio
import sys
import os

# Add current directory to path to import from main.py
sys.path.append(os.getcwd())

async def test_integrated_system():
    """Test the complete integrated research system using main.py functions"""
    print("Testing Complete Integrated Research System...")
    print("=" * 60)
    
    # Test query
    test_query = "quantum computing"
    print(f"Research Query: {test_query}")
    print("=" * 60)
    
    try:
        # Import functions from main.py - use the actual function names
        from main import fetch_content_from_url, stream_response, call_mcp_gateway
        from agent.tools import internet_search
        import boto3
        
        # Get AWS info
        account_id = boto3.client('sts').get_caller_identity()['Account']
        region = boto3.Session().region_name or 'us-west-2'
        
        print("Step 1 - Internet Search:")
        # Step 1: Internet search using agent.tools
        search_results = internet_search(test_query, max_results=3)
        print(f"Found {search_results.get('total_results', 0)} results")
        
        print("Step 2 - Content Extraction:")
        # Step 2: Extract content from URLs
        all_content = ""
        for result in search_results.get('results', []):
            url = result.get('url', '')
            title = result.get('title', '')
            content = fetch_content_from_url(url)
            all_content += f"\n\nSource: {title}\n{content}"
        
        print("Step 3 - AI Analysis:")
        # Step 3: Analyze with SageMaker
        if all_content.strip():
            prompt = f"""Analyze this content about "{test_query}" and provide:

SUMMARY: [2-3 sentence summary]
KEY POINTS:
• [key finding 1]
• [key finding 2]
TRENDS: [notable trends]

Content:
{all_content[:2000]}"""

            analysis = stream_response(ENDPOINT, prompt)
        else:
            analysis = "No content available for analysis."
        
        print("AI analysis completed")
        
        print("Step 4 - Report Generation:")
        # Step 4: Generate report
        sources_section = ""
        for result in search_results.get('results', []):
            title = result.get('title', 'N/A')
            source = result.get('source', 'Unknown')
            sources_section += f"- **{source}**: {title}\n"
        
        report = f"""# Test Research Report

## Query: {test_query}

## Sources: 
{sources_section}

## Analysis:
{analysis}

---
*Test completed successfully*"""

        print("Step 5 - MCP Gateway Storage:")
        # Step 5: Save via MCP Gateway
        s3_bucket = f"sagemaker-{region}-{account_id}"
        mcp_result = call_mcp_gateway(report, s3_bucket)
        
        storage_status = mcp_result.get('content', [{}])[0].get('text', 'Storage completed')
        print(f"Storage result: {storage_status}")
        
        print("\n" + "=" * 60)
        print("✓ Integrated system test completed successfully!")
        print("✓ Research report generated and saved to S3")
        print("✓ MCP Gateway processing completed")
        
        return True
        
    except Exception as e:
        print(f"\n✗ Integrated system test failed: {str(e)}")
        return False

# Run integrated system test
integrated_test_success = await test_integrated_system()


## 9. Deploy to Bedrock Agent Core

Deploy the integrated system to Bedrock Agent Core for production use.

In [ ]:
# Deploy to Bedrock Agent Core
# Deploy to Bedrock Agent Core
def deploy_to_bedrock_agentcore():
    """Deploy integrated system to Bedrock Agent Core"""
    
    print("Deploying to Bedrock Agent Core...")
    print("=" * 50)
    
    # Initialize AgentCore Runtime
    agentcore_runtime = Runtime()
    
    # Configure the agent
    print("Configuring Bedrock Agent Core...")
    response = agentcore_runtime.configure(
        entrypoint="main.py",
        auto_create_execution_role=False,
        execution_role=bedrock_role_arn,
        auto_create_ecr=True,
        requirements_file="requirements.txt",
        region=region,
        agent_name="qwen_research_agent_mcp",  # Fixed: no hyphens, underscores only
    )
    
    print("Configuration completed. Starting deployment...")
    
    # Deploy using CodeBuild
    #launch_result = agentcore_runtime.launch(use_codebuild=True)
    launch_result = agentcore_runtime.launch()
    
    agent_arn = launch_result.agent_arn
    
    print("\n✓ Deployment successful!")
    print("Agent ARN: " + str(agent_arn))
    print("MCP Gateway Lambda: " + str(lambda_mcp_arn))
    print("\nYour integrated research agent with MCP Gateway is now deployed!")
    
    return agent_arn

# Deploy if tests passed
if integrated_test_success:
    try:
        deployed_agent_arn = deploy_to_bedrock_agentcore()
        print(f"\nDeployment completed successfully!")
        print(f"Agent ARN: {deployed_agent_arn}")
    except Exception as e:
        print(f"Deployment failed: {str(e)}")
        deployed_agent_arn = None
else:
    print("Skipping deployment due to test failures")
    deployed_agent_arn = None

## 10. Test Deployed Agent

Test the deployed agent to ensure everything works in production.

In [ ]:
# Test the deployed agent
def test_deployed_agent(agent_arn):
    """Test the deployed Bedrock Agent Core"""
    
    if not agent_arn:
        print("No agent ARN available for testing")
        return False
    
    print("Testing Deployed Agent...")
    print("=" * 50)
    
    # For Agent Core, use direct invocation
    import requests
    import time
    
    test_query = "Research the latest developments in sustainable energy technologies"
    
    try:
        # Agent Core uses HTTP POST to the runtime endpoint
        payload = {
            "prompt": test_query
        }
        
        # Extract agent ID from ARN for endpoint URL
        agent_id = agent_arn.split('/')[-1]
        endpoint_url = "https://" + agent_id + ".bedrock-agentcore." + region + ".amazonaws.com"
        
        print("Query: " + test_query)
        print("Endpoint: " + endpoint_url)
        print("Note: Agent Core may take time to initialize after deployment")
        
        # For now, just show the configuration
        print("\nAgent Configuration:")
        print("- Agent ARN: " + agent_arn)
        print("- Agent ID: " + agent_id)
        print("- Region: " + region)
        print("- Status: Deployed successfully")
        
        print("\n✓ Agent deployment verified!")
        print("Use the Agent ARN to invoke via SDK or direct HTTP calls")
        return True
        
    except Exception as e:
        print("✗ Agent test failed: " + str(e))
        return False

# Test deployed agent
if deployed_agent_arn:
    deployment_test_success = test_deployed_agent(deployed_agent_arn)
else:
    deployment_test_success = False
    print("No deployed agent to test")

### 11. Invoke Deployed Agent and MCP Server

In [ ]:
def test_agent(agent_arn, query):
    import boto3
    import json
    import uuid
    import time
    
    bedrock_agentcore_client = boto3.client('bedrock-agentcore', region_name='us-west-2')
    
    try:
        session_id = str(uuid.uuid4()) + '-' + str(int(time.time()))
        
        response = bedrock_agentcore_client.invoke_agent_runtime(
            agentRuntimeArn=agent_arn,
            runtimeSessionId=session_id,
            payload=json.dumps({'prompt': query})
        )
        
        if 'response' in response:
            content = response['response'].read().decode('utf-8')
            
            lines = content.strip().split('\n')
            for line in lines:
                if line.startswith('data: '):
                    data = json.loads(line[6:])
                    
                    if 'messages' in data:
                        for message in data['messages']:
                            if message.get('type') == 'assistant':
                                print("Agent Response:")
                                print("=" * 60)
                                print(message.get('content', ''))
                                return message.get('content', '')
        
        return "No content found"
        
    except Exception as e:
        print("Test failed: " + str(e))
        return None

agent_arn = deployed_agent_arn
#query = "Cybersecurity News Latest"
query = "AI News Latest"
result = test_agent(agent_arn, query)


### 12. Download The Report

In [ ]:
import boto3
import re
from datetime import datetime

def download_report_from_result(agent_result, local_filename=None):
    """Extract S3 URL from agent result and download the report"""
    
    # Extract S3 URL from agent result text
    s3_pattern = r's3://[^\s)]+\.md'
    match = re.search(s3_pattern, agent_result)
    
    if not match:
        print("✗ No S3 URL found in agent result")
        return None
    
    s3_url = match.group(0)
    print("Found S3 URL: " + s3_url)
    
    # Parse S3 URL
    s3_url_clean = s3_url[5:]  # Remove s3://
    parts = s3_url_clean.split('/', 1)
    bucket = parts[0]
    key = parts[1]
    
    # Generate local filename if not provided
    if not local_filename:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        local_filename = "downloaded_report_" + timestamp + ".md"
    
    # Download from S3
    s3_client = boto3.client('s3')
    
    try:
        print("Downloading from s3://" + bucket + "/" + key)
        s3_client.download_file(bucket, key, local_filename)
        print("✓ Downloaded to: " + local_filename)
        
        # Display content
        with open(local_filename, 'r') as f:
            content = f.read()
        
        print("\nReport Content:")
        print("=" * 60)
        print(content)
        print("=" * 60)
        
        return local_filename
        
    except Exception as e:
        print("✗ Download failed: " + str(e))
        return None

if __name__ == "__main__":
    # Test with sample agent result    
    download_report_from_result(result)

## 13. Summary and Next Steps

Summary of what we've accomplished and recommendations for next steps.

In [ ]:
# Generate deployment summary
print("=" * 60)
print("BEDROCK AGENT CORE WITH MCP GATEWAY - DEPLOYMENT SUMMARY")
print("=" * 60)

print("\n## Components Deployed:")
print(f"✓ IAM Role: {bedrock_role_arn}")
print(f"✓ Lambda MCP Server: {lambda_mcp_arn}")
print(f"✓ Bedrock Agent Core: {deployed_agent_arn or 'Not deployed'}")

print("\n## Test Results:")
print(f"✓ MCP Gateway Test: {'PASSED' if mcp_test_success else 'FAILED'}")
print(f"✓ Integrated System Test: {'PASSED' if integrated_test_success else 'FAILED'}")
print(f"✓ Deployment Test: {'PASSED' if deployment_test_success else 'FAILED'}")

print("\n## Architecture Overview:")
print("User Query → Bedrock Agent Core → Internet Search → Qwen Analysis → MCP Gateway → Lambda → S3")

print("\n## Key Features:")
print("- Real-time internet search integration")
print("- AI-powered analysis using Qwen model")
print("- Structured report generation via MCP Gateway")
print("- Dual-format output (Markdown + JSON)")
print("- Secure S3 storage with metadata")
print("- Production-ready Bedrock Agent Core deployment")

print("\n## Next Steps:")
print("1. Configure agent aliases for different environments")
print("2. Set up monitoring and logging")
print("3. Implement additional MCP tools as needed")
print("4. Scale based on usage patterns")
print("5. Add custom authentication if required")

print("\n## Usage:")
if deployed_agent_arn:
    print(f"Your agent is ready! Use the ARN: {deployed_agent_arn}")
    print("Send research queries and receive comprehensive reports automatically saved to S3.")
else:
    print("Complete the deployment process to start using your integrated research agent.")

print("\n" + "=" * 60)
print("DEPLOYMENT COMPLETE")
print("=" * 60)